In [48]:
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
import numpy as np
import plotly.graph_objects as go
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling

In [60]:
class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=3, n_obj=1, n_ieq_constr=0, xl=0, xu=20, vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = 4*x[:, 0]+8*x[:, 1]+12*x[:, 2]
        out["G"] = (abs(2*x[:, 0]+x[:, 1]+5*x[:, 2]-280) + abs(6*x[:, 0]+3*x[:, 1]+12*x[:, 2]-120) +
                    abs(2*x[:, 0]+8*x[:, 1]+4*x[:, 2]-670) + abs(2*x[:, 0]+6*x[:, 1]+6*x[:, 2]-250) +
                    abs(x[:, 0]-x[:, 1]) + abs(x[:, 0]-x[:, 2]) + abs(x[:, 1]-x[:, 2]))
        
problem = MyProblem()

In [61]:
method = GA (pop_size=20,
            sampling=IntegerRandomSampling(),
            crossover=SBX(prob=1.0, eta=3, vtype=int, repair=RoundingRepair()),
            mutation=PM(prob=1.0, eta=3, vtype=int, repair=RoundingRepair()),
            eliminate_duplicates=True,
            )

In [62]:
res = minimize(problem,
               method,
               termination=('n_gen', 200),
               seed=1,
               save_history=True
               )


Exception: ('Problem Error: G can not be set, expected shape (20, 0) but provided (20,)', ValueError('cannot reshape array of size 20 into shape (20,0)'))

class ProblemWrapper(Problem):
    def _evaluate(self, designs, out, *args, **kwargs):
        res = []
        for x in designs:
            f1 = 100 * (x**2 + x**2)
            f2 = (x[:, 0]-1)**2 + x[:, 1]**2
            res.append(f1)
        out['F'] = np.array(res)

problem = ProblemWrapper(n_var=3, n_obj=2, xl=[0, -0, 0], xu=[20, 20, 20])
algorithm = NSGA2 (pop_size=500)
stop_criteria = ("n_gen", 300)

result = minimize(
    problem= problem,
    algorithm=algorithm,
    termination=stop_criteria
)
res_data = result.F.T

Imprimiendo los resultados:

fig = go.Figure(data=go.Scatter(x=res_data[0], y=res_data[1], mode='markers'))
fig.show()